<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import json
import numpy as np
import argparse
import collections
import random
import tqdm
import pandas as pd

**This article explains how squad data was collected and data description:** 
[squad Description](https://rajpurkar.github.io/mlx/qa-and-squad/) <br>
One defining characteristic of SQuAD is that the answers to all of the questions are
segments of text, or spans, in the passage. These can be single or multiple words, and are not limited to entities – any span is fair game.

SQuAD uses two different metrics to evaluate how well a system does on the benchmark. The Exact Match metric measures the percentage of predictions that match any one of the ground truth answers exactly. The F1 score metric is a looser metric measures the average overlap between the prediction and ground truth answer.

We first assess human performance on SQuAD. To evaluate human performance, we treat the one of the crowdsourced answers as the human prediction, and keep the other answers as ground truth answers. The resulting human performance score on the test set is 82.3% for the exact match metric, and 91.2% F1.

**Difference between V2 and V1:** SQuAD2.0 combines the 100,000 questions in SQuAD1.1 with over 50,000 new, unanswerable questions written adversarially by crowdworkers to look similar to answerable ones. 

**To see models performance on squad: [Official Sqaud site](https://rajpurkar.github.io/SQuAD-explorer/explore/v2.0/dev/)**

In [94]:
# Read the train data
data = pd.read_json("data/train-v2.0.json")['data']

In [100]:
def random_selection(data, seed):
    '''
    This function will select a random subject since it's the first key, 
    then it will choose a random tuple of qas and context from all the possible tuples.
    '''
    length = len(random.choice(data)['paragraphs'])
    random.seed(seed)
    choosed_data = random.choice(data)
    choosed_index = random.randint(0, len(choosed_data['paragraphs']))
    return {'title':choosed_data['title'], 'qas': choosed_data['paragraphs']
                      [choosed_index]['qas'],
          'context': choosed_data['paragraphs'][choosed_index]['context']}

In [101]:
random_selection(data, 0)

{'title': 'University_of_Notre_Dame',
 'qas': [{'question': 'How many colleges for undergraduates are at Notre Dame?',
   'id': '5733a70c4776f41900660f62',
   'answers': [{'text': 'five', 'answer_start': 68}],
   'is_impossible': False},
  {'question': 'What was created at Notre Dame in 1962 to assist first year students?',
   'id': '5733a70c4776f41900660f63',
   'answers': [{'text': 'The First Year of Studies program',
     'answer_start': 155}],
   'is_impossible': False},
  {'question': 'What entity provides help with the management of time for new students at Notre Dame?',
   'id': '5733a70c4776f41900660f64',
   'answers': [{'text': 'Learning Resource Center', 'answer_start': 496}],
   'is_impossible': False},
  {'question': 'Which organization declared the First Year of Studies program at Notre Dame "outstanding?"',
   'id': '5733a70c4776f41900660f65',
   'answers': [{'text': 'U.S. News & World Report', 'answer_start': 647}],
   'is_impossible': False}],
 'context': "All of Notre 

**Perfect correct answer rate! The text is descriptive text of the university of notre dame. The questions are kind of straightforward. Sentences in the context contain the same key words as in questions so maybe this explains why we have no errors.**

In [102]:
random_selection(data, 1)

{'title': 'Dialect',
 'qas': [{'question': 'After what war did Moscow begin to repress the Ukrainian language?',
   'id': '56e8368700c9c71400d7765b',
   'answers': [{'text': 'World War II', 'answer_start': 6}],
   'is_impossible': False},
  {'question': 'Why did Moscow begin to repress the Ukrainian language?',
   'id': '56e8368700c9c71400d7765c',
   'answers': [{'text': 'Ukrainian collaborationism with the Axis powers',
     'answer_start': 27}],
   'is_impossible': False},
  {'question': 'Why did Ukrainians collaborate with the Axis?',
   'id': '56e8368700c9c71400d7765d',
   'answers': [{'text': 'to gain independence', 'answer_start': 89}],
   'is_impossible': False},
  {'plausible_answers': [{'text': 'changed its policy towards repression of the Ukrainian language.',
     'answer_start': 118}],
   'question': 'What happened to Moscow policy after World War I?',
   'id': '5ad27f44d7d075001a42975c',
   'answers': [],
   'is_impossible': True},
  {'plausible_answers': [{'text': 'After 

**Even though the length of the context is too small! Just one sentence, we can see that the question 'When did the Ukraine change its policy towards repression of the Moscow language?' is classed as non answerable questions but the answer exists and it's put in the plausible answers instead of answers.<br>
Also the last questions, so for this chunk of data we can say they we have the adequate responses.**

In [103]:
random_selection(data, 2)

{'title': 'Matter',
 'qas': [{'plausible_answers': [{'text': 'four', 'answer_start': 42}],
   'question': 'How many quarks and leptons are there?',
   'id': '5a7e05ef70df9f001a875425',
   'answers': [],
   'is_impossible': True},
  {'plausible_answers': [{'text': 'The Standard Model', 'answer_start': 138}],
   'question': 'What model satisfactorily explains gravity?',
   'id': '5a7e05ef70df9f001a875426',
   'answers': [],
   'is_impossible': True},
  {'plausible_answers': [{'text': 'force-carrying particles',
     'answer_start': 472}],
   'question': 'Interactions between quarks and leptons are the exchange of what?',
   'id': '5a7e05ef70df9f001a875427',
   'answers': [],
   'is_impossible': True},
  {'plausible_answers': [{'text': 'matter', 'answer_start': 711}],
   'question': 'Mass and energy can always be compared to what?',
   'id': '5a7e05ef70df9f001a875428',
   'answers': [],
   'is_impossible': True},
  {'plausible_answers': [{'text': 'Planck', 'answer_start': 953}],
   'quest

**Here we can see that there is answers for all the questions giving that the size of the context is big compared to other contexts. Here some of the plausible answers are correct! but we can see that the text is about physics so maybe it's not obvious like other texts.<br>
So if the model do not predict the correct answers for this kind of subjects it's normal since people who entered the responses does know the correct answers.**

In [104]:
random_selection(data, 3)

{'title': 'Emotion',
 'qas': [{'question': 'What do psychologists use to visually map emotional experiences?',
   'id': '570a7cdb6d058f1900182ea0',
   'answers': [{'text': 'multidimensional scaling', 'answer_start': 19}],
   'is_impossible': False},
  {'question': 'Along with valence, what is the other of the two dimensions used to map emotional experiences?',
   'id': '570a7cdb6d058f1900182ea1',
   'answers': [{'text': 'arousal', 'answer_start': 403}],
   'is_impossible': False},
  {'question': 'What is the term for the feeling of energy or enervation created by an emotional experience?',
   'id': '570a7cdb6d058f1900182ea2',
   'answers': [{'text': 'arousal', 'answer_start': 403}],
   'is_impossible': False},
  {'question': 'What is the term for the degree to which an emotional experience feels positive or negative?',
   'id': '570a7cdb6d058f1900182ea3',
   'answers': [{'text': 'valence', 'answer_start': 343}],
   'is_impossible': False},
  {'question': 'What component of emotion prov

**The text is kind of hard since it talks about psychology experiences, however when we use the negation in the questions people get confused!**

In [105]:
random_selection(data, 4)

{'title': 'Hanover',
 'qas': [{'question': 'What is a Schnellweg?',
   'id': '570a01776d058f1900182c5e',
   'answers': [{'text': 'expressway', 'answer_start': 20}],
   'is_impossible': False},
  {'question': 'What structure does the Schnellweg and a number of roads roughly form?',
   'id': '570a01776d058f1900182c5f',
   'answers': [{'text': 'a large ring road', 'answer_start': 109}],
   'is_impossible': False},
  {'question': 'When does the Messeschnellweg change to the B6?',
   'id': '570a01776d058f1900182c60',
   'answers': [{'text': 'Seelhorster Kreuz', 'answer_start': 354}],
   'is_impossible': False},
  {'question': 'What road does the Messeschnellweg pass the Hanover fairground as?',
   'id': '570a01776d058f1900182c61',
   'answers': [{'text': 'B6', 'answer_start': 411}],
   'is_impossible': False},
  {'question': 'What road does the Südschnellweg start out as?',
   'id': '570a01776d058f1900182c62',
   'answers': [{'text': 'B65', 'answer_start': 493}],
   'is_impossible': False},

**Here we can see that the text is too much complex even for human (me). To keep track of all the links is hard.**

In [106]:
random_selection(data, 5)

{'title': 'Renewable_energy_commercialization',
 'qas': [{'question': 'How is the wind industry in the USA able to produce more power at lower cost?',
   'id': '572950ad6aef051400154cb4',
   'answers': [{'text': 'by using taller wind turbines with longer blades',
     'answer_start': 85}],
   'is_impossible': False},
  {'question': 'How tall are new wind turbines that are being built in Indiana, Michigan and Ohio?',
   'id': '572950ad6aef051400154cb5',
   'answers': [{'text': '300 feet to 400 feet above the ground',
     'answer_start': 301}],
   'is_impossible': False},
  {'question': 'Wind turbines in the USA can now compete with what type of fuel?',
   'id': '572950ad6aef051400154cb6',
   'answers': [{'text': 'conventional fossil fuels like coal',
     'answer_start': 360}],
   'is_impossible': False},
  {'question': 'Prices have fallen as low as what price?',
   'id': '572950ad6aef051400154cb7',
   'answers': [{'text': '4 cents per kilowatt-hour', 'answer_start': 425}],
   'is_impo

In [107]:
random_selection(data, 6)

{'title': 'Bird',
 'qas': [{'question': 'In which type of migration do birds use the sun to navigate by day and a stellar compass at night?',
   'id': '5731e1c3b9d445190005e607',
   'answers': [{'text': 'diurnal migrants', 'answer_start': 337}],
   'is_impossible': False},
  {'question': 'Some species use specialised photoreceptors to sense what?',
   'id': '5731e1c3b9d445190005e608',
   'answers': [{'text': "the Earth's geomagnetism", 'answer_start': 716}],
   'is_impossible': False},
  {'question': 'What do birds to compensate for the changing position of the sun during the day?',
   'id': '5731e1c3b9d445190005e609',
   'answers': [{'text': 'internal clock', 'answer_start': 534}],
   'is_impossible': False}],
 'context': "The ability of birds to return to precise locations across vast distances has been known for some time; in an experiment conducted in the 1950s a Manx shearwater released in Boston returned to its colony in Skomer, Wales, within 13 days, a distance of 5,150 km (3,20